#Fine-tuning DistilBERT on AG News with MLflow
distilbert-base-uncased — це стиснена версія моделі BERT, створена компанією Hugging Face.

Основні характеристики:
* Архітектура: DistilBERT (компресований BERT з менше шарів)

* Без регістрочутливості (uncased): розглядає "Apple" і "apple" як однакові

* Розмір: ~66M параметрів (легше і швидше, ніж BERT)

* Підходить для задач класифікації тексту, питання-відповідь, тощо

* Використовується часто як базова модель для fine-tuning

Чому обрана?
* Швидко тренується навіть на CPU або маленькому GPU

* Широко підтримується в екосистемі transformers

* Підтримується MLflow через mlflow.transformers

Датасет: ag_news
AG News — один із класичних датасетів для задачі класифікації новинних текстів.

Структура: 120K тренувальних прикладів
4 класи:
* World
* Sports
* Business
* Sci/Tech

* Кожен приклад — це пара (заголовок, опис) новинної статті + мітка
* Доступний напряму з Hugging Face Datasets (load_dataset("ag_news"))
* Добре підходить для демонстрації багатокласової класифікації
* Навіть 1% (~1200 прикладів) вистачає для тренування демо-моделі
* Стандартний для бенчмаркінгу NLP-моделей


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Етап 1: Встановлення бібліотек

In [8]:
!pip install mlflow scikit-learn

In [ ]:
!pip install transformers

##Етап 2: Завантаження датасету

In [ ]:
!pip install -U datasets

In [9]:
from datasets import load_dataset
import pandas as pd

# Завантаження датасету
dataset = load_dataset("wangrongsheng/ag_news", download_mode="force_redownload")

# Вибірка невеликої підмножини для тренування і тесту
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(12))  # 12 для візуалізації
small_test_dataset = dataset["test"].shuffle(seed=42).select(range(5))

# Конвертуємо у pandas DataFrame для гарного відображення
train_df = pd.DataFrame(small_train_dataset)
test_df = pd.DataFrame(small_test_dataset)

print("Train subset:")
display(train_df)

print("\nTest subset:")
display(test_df)


data/train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Train subset:


,text,label
0,Bangladesh paralysed by strikes Opposition act...,0
1,Desiring Stability Redskins coach Joe Gibbs ex...,1
2,Will Putin #39;s Power Play Make Russia Safer?...,0
3,U2 pitches for Apple New iTunes ads airing dur...,3
4,S African TV in beheading blunder Public broad...,0
5,A Cosmic Storm: When Galaxy Clusters Collide A...,3
6,West sets deadline for Iran to freeze uranium ...,0
7,Computer Assoc. Cuts 800 Jobs Worldwide (AP) A...,3
8,CA Opens Utility Pricing for Mainframes Keepin...,3
9,Economy builds steam in KC Fed district The ec...,2



Test subset:


,text,label
0,Indian board plans own telecast of Australia s...,1
1,Stocks Higher on Drop in Jobless Claims A shar...,2
2,"Nuggets 112, Raptors 106 Carmelo Anthony score...",1
3,Stocks Higher on Drop in Jobless Claims A shar...,2
4,REVIEW: 'Half-Life 2' a Tech Masterpiece (AP) ...,3


##Етап 3: Попередня обробка

In [10]:
from transformers import DistilBertTokenizerFast

# Ініціалізуємо токенізатор для DistilBERT
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

# Функція токенізації: приймає приклад (рядок тексту) і повертає токенізовані дані
def tokenize_function(example):
    return tokenizer(
        example["text"],           # текст для токенізації
        padding="max_length",      # додаємо паддінг до максимальної довжини послідовності
        truncation=True            # обрізаємо тексти довші за максимальну довжину
    )

# Застосовуємо токенізацію до всього набору тренувальних даних
tokenized_train = small_train_dataset.map(tokenize_function, batched=True)

# Аналогічно для тестового набору
tokenized_test = small_test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/12 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

 ## Етап 4: Підготовка моделі

In [11]:
from transformers import DistilBertForSequenceClassification

# Завантажуємо попередньо натреновану модель DistilBERT для задачі класифікації
# Вказуємо кількість класів — 4 (для AG News)
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=4
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Етап 5: Тренування з MLflow

In [12]:
# Імпортуємо необхідні бібліотеки
import mlflow
import mlflow.transformers
from transformers import Trainer, TrainingArguments

# Налаштування параметрів тренування
training_args = TrainingArguments(
    output_dir="./results",                       # Каталог для збереження результатів
    learning_rate=2e-5,                          # Швидкість навчання
    per_device_train_batch_size=8,               # Розмір батчу для тренування
    per_device_eval_batch_size=8,                # Розмір батчу для валідації
    num_train_epochs=3,                          # Кількість епох
    weight_decay=0.01,                           # Зменшення ваг (регуляризація)
    logging_dir="./logs",                        # Каталог для логів
    logging_steps=10,                            # Кожні 10 кроків — логування
)

# Ініціалізуємо Trainer з моделлю та даними
trainer = Trainer(
    model=model,                                 # Модель для тренування (DistilBERT)
    args=training_args,                          # Аргументи тренування
    train_dataset=tokenized_train,               # Тренувальна вибірка
    eval_dataset=tokenized_test,                 # Валідаційна вибірка
)

# Автоматичне логування параметрів, метрик і моделі за допомогою MLflow
mlflow.transformers.autolog()

# Починаємо сесію логування з MLflow
with mlflow.start_run():
    trainer.train()                              # Запускаємо тренування


2025/07/24 17:24:48 WARNING mlflow.utils.autologging_utils: MLflow transformers autologging is known to be compatible with 4.35.2 <= transformers <= 4.52.4, but the installed version is 4.53.2. If you encounter errors during autologging, try upgrading / downgrading transformers to a compatible version, or try upgrading MLflow.


Step,Training Loss


In [13]:
trainer.evaluate()  # Оцінюємо модель на валідаційній вибірці та виводимо метрики
trainer.save_model("distilbert-agnews")          # Зберігаємо модель у папку "distilbert-agnews"
tokenizer.save_pretrained("distilbert-agnews")   # Зберігаємо токенізатор у ту саму папку


('distilbert-agnews/tokenizer_config.json',
 'distilbert-agnews/special_tokens_map.json',
 'distilbert-agnews/vocab.txt',
 'distilbert-agnews/added_tokens.json',
 'distilbert-agnews/tokenizer.json')